In [1]:
# code below works marvelously it does store the embedding nicely dont change it 

In [4]:
import cv2
import face_recognition
import numpy as np
import pymysql


def insert_face_embedding(cursor, account_number, embedding):
    # Insert the face embedding into the face table along with the account number
    cursor.execute('INSERT INTO face_db (account_number, face_embedding) VALUES (%s, %s)', (account_number, embedding))

def capture_and_store_faces(cursor, connection, account_number, num_faces):
    # Start the webcam for capturing images
    video_capture = cv2.VideoCapture(0)

    face_count = 0

    while face_count < num_faces:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Convert the image from BGR color (OpenCV default) to RGB color
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb_image)

        # If no face is detected, continue to the next frame
        if len(face_locations) == 0:
            continue

        # Encode the detected face images
        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

        for face_encoding in face_encodings:
            # Store the face embedding in the database
            insert_face_embedding(cursor, account_number, np.array(face_encoding))

            face_count += 1

        # Display the captured image with bounding boxes
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        cv2.imshow('Video', frame)
        cv2.waitKey(1)

    # Release the webcam and close the OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()
    connection.commit()


In [5]:
# Example usage
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='52456',
                             db='mydatabase',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
  # Create the face table if it doesn't exist

    account_number = 129098788
    num_faces = 10

    # Capture and store faces in the face database for the specified account number
    capture_and_store_faces(cursor, connection, account_number, num_faces)

connection.close()  # Close the database connection